# Ex2 - Getting and Knowing your Data

Check out [Chipotle Exercises Video Tutorial](https://www.youtube.com/watch?v=lpuYZ5EUyS8&list=PLgJhDSE2ZLxaY_DigHeiIDC1cD09rXgJv&index=2) to watch a data scientist go through the exercises

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [42]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ChipotleSQL').getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).

Pandas way:

In [43]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'

chipo = pd.read_csv(url, sep = '\t')

Pyspark Way:


In [44]:
chipo = spark.createDataFrame(chipo_pd)

In [45]:
from pyspark.sql.functions import regexp_replace

In [47]:
df = chipo.withColumn('item_price', regexp_replace('item_price', '[$]', '').cast('double'))

In [49]:
df.createOrReplaceTempView('df')

### Step 3. Assign it to a variable called chipo.

### Step 4. See the first 10 entries

In [51]:

spark.sql('select * from df limit 10').show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                 NaN|      2.39|
|       1|       1|                Izze|        [Clementine]|      3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39|
|       1|       1|Chips and Tomatil...|                 NaN|      2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|
|       3|       1|       Side of Chips|                 NaN|      1.69|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|     11.75|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|      9.25|
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|      9.25|
+--------+--------+--------------------+-----------

In [ ]:
chipo.show(10, truncate = True)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                 NaN|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                 NaN|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                 NaN|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [53]:
spark.sql('select count(*) as total_rows from df').show(truncate = True)

+----------+
|total_rows|
+----------+
|      4622|
+----------+



In [ ]:
chipo.count()

4622

In [ ]:
# Solution 2

chipo.info() # entries <= 4622 observations

PySparkAttributeError: [ATTRIBUTE_NOT_SUPPORTED] Attribute `info` is not supported.

### Step 6. What is the number of columns in the dataset?

In [ ]:
chipo.count()

4622

In [56]:
query1 = 'DESCRIBE df'

spark.sql(query1).show(truncate = True)

+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|          order_id|   bigint|   NULL|
|          quantity|   bigint|   NULL|
|         item_name|   string|   NULL|
|choice_description|   string|   NULL|
|        item_price|   double|   NULL|
+------------------+---------+-------+



### Step 7. Print the name of all the columns.

In [ ]:
chipo.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. How is the dataset indexed?

In [ ]:
chipo.rdd.getNumPartitions()

2

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
chipo = chipo.withColumn("id", monotonically_increasing_id())

In [ ]:
chipo.show(10, truncate = True)


+--------+--------+--------------------+--------------------+----------+---+
|order_id|quantity|           item_name|  choice_description|item_price| id|
+--------+--------+--------------------+--------------------+----------+---+
|       1|       1|Chips and Fresh T...|                 NaN|    $2.39 |  0|
|       1|       1|                Izze|        [Clementine]|    $3.39 |  1|
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |  2|
|       1|       1|Chips and Tomatil...|                 NaN|    $2.39 |  3|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |  4|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |  5|
|       3|       1|       Side of Chips|                 NaN|    $1.69 |  6|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |  7|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |  8|
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |  9|

### Step 9. Which was the most-ordered item?

In [ ]:
from pyspark.sql.functions import sum as _sum

In [ ]:


select item_name, sum(quantity) as total_quantity
from df
group by item_name
order by total_quantity desc

In [59]:
query1 = '''
select item_name, sum(quantity)
from df
group by item_name
order by sum(quantity) desc
limit 1
'''

spark.sql(query1).show(truncate = True)

+------------+-------------+
|   item_name|sum(quantity)|
+------------+-------------+
|Chicken Bowl|          761|
+------------+-------------+



In [ ]:
chipo.groupBy('item_name') \
    .agg(_sum('quantity').alias('total_quantity')) \
    .orderBy('total_quantity', ascending = False).show(10, truncate= True)


+--------------------+--------------+
|           item_name|total_quantity|
+--------------------+--------------+
|        Chicken Bowl|           761|
|     Chicken Burrito|           591|
| Chips and Guacamole|           506|
|       Steak Burrito|           386|
|   Canned Soft Drink|           351|
|               Chips|           230|
|          Steak Bowl|           221|
|       Bottled Water|           211|
|Chips and Fresh T...|           130|
|         Canned Soda|           126|
+--------------------+--------------+
only showing top 10 rows


In [ ]:
c = chipo.groupby('item_name')
c = c.sum()
c = c.sort_values(['quantity'], ascending=False)
c.head(1)

,order_id,quantity
item_name,,
Chicken Bowl,713926,761


### Step 10. For the most-ordered item, how many items were ordered?

In [60]:
query1 = '''
select sum(quantity)
from df
group by item_name
order by sum(quantity) desc
limit 1
'''

spark.sql(query1).show(truncate = True)

+-------------+
|sum(quantity)|
+-------------+
|          761|
+-------------+



In [ ]:
chipo.groupBy('item_name') \
    .agg(_sum('quantity').alias('total_quantity')) \
    .orderBy('total_quantity', ascending = False).show(10, truncate= True)('total_quanity')


+--------------------+--------------+
|           item_name|total_quantity|
+--------------------+--------------+
|        Chicken Bowl|           761|
|     Chicken Burrito|           591|
| Chips and Guacamole|           506|
|       Steak Burrito|           386|
|   Canned Soft Drink|           351|
|               Chips|           230|
|          Steak Bowl|           221|
|       Bottled Water|           211|
|Chips and Fresh T...|           130|
|         Canned Soda|           126|
+--------------------+--------------+
only showing top 10 rows


TypeError: 'NoneType' object is not callable

In [ ]:
c = chipo.groupby('item_name')
c = c.sum()
c = c.sort_values(['quantity'], ascending=False)
c.head(1)

,order_id,quantity
item_name,,
Chicken Bowl,713926,761


### Step 11. What was the most ordered item in the choice_description column?

In [62]:
query1 = '''
select choice_description, sum(quantity)
from df
group by choice_description
order by sum(quantity) desc
limit 2
'''

spark.sql(query1).show(truncate = True)

+------------------+-------------+
|choice_description|sum(quantity)|
+------------------+-------------+
|               NaN|         1382|
|       [Diet Coke]|          159|
+------------------+-------------+



In [ ]:
chipo.show(10, truncate = True)

+--------+--------+--------------------+--------------------+----------+---+
|order_id|quantity|           item_name|  choice_description|item_price| id|
+--------+--------+--------------------+--------------------+----------+---+
|       1|       1|Chips and Fresh T...|                 NaN|    $2.39 |  0|
|       1|       1|                Izze|        [Clementine]|    $3.39 |  1|
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |  2|
|       1|       1|Chips and Tomatil...|                 NaN|    $2.39 |  3|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |  4|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |  5|
|       3|       1|       Side of Chips|                 NaN|    $1.69 |  6|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |  7|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |  8|
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |  9|

In [ ]:
from pyspark.sql.functions import mean as _mean, min as _min, max as _max
chipo.groupBy('choice_description').agg(_sum('quantity').alias('cd_total_quantity')).orderBy('cd_total_quantity', ascending = False).show(2)

+------------------+-----------------+
|choice_description|cd_total_quantity|
+------------------+-----------------+
|               NaN|             1382|
|       [Diet Coke]|              159|
+------------------+-----------------+
only showing top 2 rows


In [ ]:
c = chipo_pd.groupby('choice_description').sum()
c = c.sort_values(['quantity'], ascending=False)
c.head(1)
# Diet Coke 159

,order_id,quantity,item_name,item_price
choice_description,,,,
[Diet Coke],123455,159,Canned SodaCanned SodaCanned Soda6 Pack Soft D...,$2.18 $1.09 $1.09 $6.49 $2.18 $1.25 $1.09 $6.4...


### Step 12. How many items were orderd in total?

In [ ]:
total_items_orders = chipo.quantity.sum()
total_items_orders

4972

In [64]:
query1 = '''

select sum(quantity)
from df
'''


spark.sql(query1).show(truncate = True)



+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [ ]:
chipo.item_price.dtype

dtype('O')

#### Step 13.b. Create a lambda function and change the type of item price

In [ ]:
dollarizer = lambda x: float(x[1:-1])
chipo.item_price = chipo.item_price.apply(dollarizer)

#### Step 13.c. Check the item price type

In [ ]:
chipo.item_price.dtype

dtype('float64')

### Step 14. How much was the revenue for the period in the dataset?

In [ ]:
revenue = (chipo['quantity']* chipo['item_price']).sum()

print('Revenue was: $' + str(np.round(revenue,2)))

Revenue was: $39237.02


In [66]:
query1 = '''
select sum(item_price*quantity)
from df
'''

spark.sql(query1).show(truncate = True)

+----------------------------+
|sum((item_price * quantity))|
+----------------------------+
|          39237.020000000084|
+----------------------------+



### Step 15. How many orders were made in the period?

In [ ]:
orders = chipo.order_id.value_counts().count()
orders

1834

In [70]:
query1 = '''
select count(distinct order_id)
from df
'''

spark.sql(query1).show(truncate = True)

+------------------------+
|count(DISTINCT order_id)|
+------------------------+
|                    1834|
+------------------------+



### Step 16. What is the average revenue amount per order?

In [80]:
query1 = '''
select avg(avg_rev)
from (
select order_id, sum(quantity*item_price) as avg_rev
from df
group by order_id)
'''

spark.sql(query1).show(truncate = True)

+------------------+
|      avg(avg_rev)|
+------------------+
|21.394231188658722|
+------------------+



In [ ]:
# Solution 1

chipo['revenue'] = chipo['quantity'] * chipo['item_price']
order_grouped = chipo.groupby(by=['order_id']).sum()
order_grouped.mean()['revenue']

21.394231188658654

In [ ]:
# Solution 2

chipo.groupby('order_id')['revenue'].sum().mean()

21.394231188658654

### Step 17. How many different items are sold?

In [ ]:
chipo.item_name.value_counts().count()

50

In [90]:
query1 = '''
select count(item_name)
from (
select distinct(item_name)
from df)
'''

spark.sql(query1).show(30,truncate = False)

+----------------+
|count(item_name)|
+----------------+
|50              |
+----------------+



In [91]:
query1 = '''
select count(distinct item_name)
from df
'''

spark.sql(query1).show(30,truncate = False)

+-------------------------+
|count(DISTINCT item_name)|
+-------------------------+
|50                       |
+-------------------------+

